# Notebook 5: Vòng Lặp Phản Hồi & Tái Tạo CoT

**Mục tiêu**: Xây dựng vòng lặp tự động cải thiện dựa trên feedback từ DeepEval

Trong notebook này, chúng ta sẽ học cách:
1. Xây dựng automated feedback pipeline với 5 giai đoạn
2. Triển khai advanced feedback strategies
3. Tạo pattern recognition cho failures
4. Ứng dụng thực tiễn với real-world case studies

---

## 🛠️ Cài Đặt và Import

Trước tiên, hãy import các thư viện cần thiết và thiết lập môi trường.

In [ ]:
import os
import json
import time
import logging
from typing import List, Dict, Any, Optional, Tuple
from dataclasses import dataclass, asdict
from datetime import datetime
import pandas as pd
import numpy as np
from collections import defaultdict, Counter
import re

# DeepEval imports
from deepeval import evaluate, assert_test
from deepeval.test_case import LLMTestCase
from deepeval.metrics import (
    AnswerRelevancyMetric,
    HallucinationMetric,
    GEval,
    BaseMetric
)

# LangGraph imports
from langgraph.graph import StateGraph, END
from langgraph.graph.message import MessagesState
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate

# Logging setup
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print("✅ Các thư viện đã được import thành công!")

## 🔑 Thiết Lập API Keys

Đảm bảo bạn đã thiết lập các API keys cần thiết.

In [ ]:
# Kiểm tra API keys
if not os.getenv("OPENAI_API_KEY"):
    print("⚠️ Chưa thiết lập OPENAI_API_KEY")
    print("Hãy thiết lập: export OPENAI_API_KEY='your-api-key'")
else:
    print("✅ OpenAI API key đã được thiết lập")

# Initialize LLM
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1,
    max_tokens=2000
)

print("✅ LLM đã được khởi tạo thành công!")

## 🧠 Custom Metrics cho CoT Evaluation

Đầu tiên, chúng ta sẽ tạo các custom metrics để đánh giá Chain-of-Thought reasoning.

In [ ]:
class LogicalFlowMetric(BaseMetric):
    """Đánh giá chuỗi suy luận logic trong CoT"""
    
    def __init__(self, threshold: float = 0.7):
        self.threshold = threshold
    
    def measure(self, test_case: LLMTestCase) -> float:
        evaluation_prompt = f"""
        Đánh giá tính logic và mạch lạc của chuỗi suy luận sau đây:
        
        Câu hỏi: {test_case.input}
        Chuỗi suy luận: {test_case.actual_output}
        
        Tiêu chí đánh giá:
        1. Tính logic: Các bước có theo thứ tự hợp lý không?
        2. Tính mạch lạc: Có mối liên kết rõ ràng giữa các bước?
        3. Tính đầy đủ: Có bỏ qua bước quan trọng nào không?
        4. Tính chính xác: Kết luận có phù hợp với quá trình suy luận?
        
        Cho điểm từ 0-10 (10 = hoàn hảo, 0 = hoàn toàn thiếu logic).
        Chỉ trả về số điểm, không giải thích.
        """
        
        response = llm.invoke(evaluation_prompt)
        try:
            score = float(response.content.strip()) / 10.0
            self.score = score
            self.reason = f"Logic flow score: {score:.2f}"
            self.success = score >= self.threshold
            return score
        except ValueError:
            self.score = 0.0
            self.reason = "Không thể đánh giá logic flow"
            self.success = False
            return 0.0
    
    def is_successful(self) -> bool:
        return self.success

class StepCompletenessMetric(BaseMetric):
    """Đánh giá tính đầy đủ của các bước trong CoT"""
    
    def __init__(self, threshold: float = 0.8):
        self.threshold = threshold
    
    def measure(self, test_case: LLMTestCase) -> float:
        evaluation_prompt = f"""
        Đánh giá tính đầy đủ của các bước giải quyết vấn đề:
        
        Vấn đề: {test_case.input}
        Các bước được thực hiện: {test_case.actual_output}
        
        Tiêu chí:
        1. Có xác định rõ vấn đề?
        2. Có phân tích các yếu tố liên quan?
        3. Có đề xuất giải pháp cụ thể?
        4. Có kiểm tra tính khả thi?
        5. Có kết luận rõ ràng?
        
        Cho điểm từ 0-10 dựa trên số bước thiết yếu được thực hiện.
        Chỉ trả về số điểm.
        """
        
        response = llm.invoke(evaluation_prompt)
        try:
            score = float(response.content.strip()) / 10.0
            self.score = score
            self.reason = f"Step completeness: {score:.2f}"
            self.success = score >= self.threshold
            return score
        except ValueError:
            self.score = 0.0
            self.reason = "Không thể đánh giá step completeness"
            self.success = False
            return 0.0
    
    def is_successful(self) -> bool:
        return self.success

print("✅ Custom metrics đã được tạo thành công!")

## 🔄 Core Feedback Pipeline Classes

Bây giờ chúng ta sẽ tạo các class cốt lõi cho feedback pipeline.

In [ ]:
@dataclass
class FeedbackResult:
    """Lưu trữ kết quả của một lần feedback"""
    iteration: int
    original_prompt: str
    enhanced_prompt: str
    original_output: str
    enhanced_output: str
    original_scores: Dict[str, float]
    enhanced_scores: Dict[str, float]
    improvement: Dict[str, float]
    feedback_used: List[str]
    timestamp: str
    success: bool

@dataclass
class FailurePattern:
    """Mô tả pattern của failure"""
    pattern_type: str
    description: str
    frequency: int
    suggested_fix: str
    examples: List[str]

class FeedbackExtractor:
    """Trích xuất feedback từ failed evaluations"""
    
    def __init__(self, llm):
        self.llm = llm
    
    def extract_feedback(self, test_case: LLMTestCase, failed_metrics: List[BaseMetric]) -> List[str]:
        """Trích xuất feedback từ các metrics bị fail"""
        feedback_list = []
        
        for metric in failed_metrics:
            if hasattr(metric, 'reason') and metric.reason:
                # Tạo feedback chi tiết hơn
                detailed_feedback = self._generate_detailed_feedback(
                    test_case, metric
                )
                feedback_list.append(detailed_feedback)
        
        return feedback_list
    
    def _generate_detailed_feedback(self, test_case: LLMTestCase, metric: BaseMetric) -> str:
        """Tạo feedback chi tiết cho metric cụ thể"""
        feedback_prompt = f"""
        Một AI agent đã thất bại trong đánh giá sau:
        
        Input: {test_case.input}
        Output: {test_case.actual_output}
        Metric: {metric.__class__.__name__}
        Reason: {getattr(metric, 'reason', 'Unknown')}
        Score: {getattr(metric, 'score', 'Unknown')}
        
        Hãy phân tích vấn đề và đưa ra feedback cụ thể để cải thiện:
        1. Vấn đề chính là gì?
        2. Làm thế nào để sửa?
        3. Ví dụ cụ thể về cách cải thiện?
        
        Trả lời ngắn gọn, tập trung vào hành động cụ thể.
        """
        
        response = self.llm.invoke(feedback_prompt)
        return response.content.strip()

class PromptEnhancer:
    """Cải thiện prompt dựa trên feedback"""
    
    def __init__(self, llm):
        self.llm = llm
    
    def enhance_prompt(self, original_prompt: str, feedback_list: List[str]) -> str:
        """Cải thiện prompt dựa trên feedback"""
        enhancement_prompt = f"""
        Bạn là một chuyên gia prompt engineering. Hãy cải thiện prompt sau dựa trên feedback:
        
        PROMPT GỐC:
        {original_prompt}
        
        FEEDBACK:
        {chr(10).join(f"- {fb}" for fb in feedback_list)}
        
        Hãy tạo prompt mới:
        1. Giữ nguyên ý nghĩa chính
        2. Tích hợp feedback để khắc phục vấn đề
        3. Thêm hướng dẫn cụ thể cho Chain-of-Thought
        4. Đảm bảo prompt rõ ràng và có cấu trúc
        
        CHỈ TRẢ VỀ PROMPT MỚI, KHÔNG GIẢI THÍCH.
        """
        
        response = self.llm.invoke(enhancement_prompt)
        return response.content.strip()

print("✅ Feedback pipeline classes đã được tạo!")

## 🚀 Automated Feedback Pipeline

Đây là phần cốt lõi - pipeline tự động với 5 giai đoạn.

In [ ]:
class AutomatedFeedbackPipeline:
    """Pipeline tự động cho feedback loop"""
    
    def __init__(self, llm, max_iterations: int = 3):
        self.llm = llm
        self.max_iterations = max_iterations
        self.feedback_extractor = FeedbackExtractor(llm)
        self.prompt_enhancer = PromptEnhancer(llm)
        self.results: List[FeedbackResult] = []
        self.failure_patterns: List[FailurePattern] = []
    
    def run_pipeline(self, 
                    initial_prompt: str, 
                    test_input: str,
                    metrics: List[BaseMetric],
                    context: Optional[str] = None) -> FeedbackResult:
        """Chạy toàn bộ pipeline feedback"""
        
        current_prompt = initial_prompt
        iteration = 0
        
        print(f"🚀 Bắt đầu Automated Feedback Pipeline")
        print(f"📝 Test input: {test_input[:100]}...")
        print(f"📊 Số metrics: {len(metrics)}")
        print("="*50)
        
        while iteration < self.max_iterations:
            iteration += 1
            print(f"\n🔄 ITERATION {iteration}")
            
            # Phase 1: Execution
            print("1️⃣ Execution Phase...")
            output = self._execute_prompt(current_prompt, test_input)
            
            # Phase 2: Evaluation
            print("2️⃣ Evaluation Phase...")
            test_case = LLMTestCase(
                input=test_input,
                actual_output=output,
                context=[context] if context else None
            )
            
            scores, failed_metrics = self._evaluate_test_case(test_case, metrics)
            
            # Kiểm tra nếu đã pass tất cả metrics
            if not failed_metrics:
                print("✅ Tất cả metrics đã pass!")
                success_result = FeedbackResult(
                    iteration=iteration,
                    original_prompt=initial_prompt,
                    enhanced_prompt=current_prompt,
                    original_output="N/A",
                    enhanced_output=output,
                    original_scores={},
                    enhanced_scores=scores,
                    improvement=scores,
                    feedback_used=[],
                    timestamp=datetime.now().isoformat(),
                    success=True
                )
                self.results.append(success_result)
                return success_result
            
            # Phase 3: Feedback Extraction
            print("3️⃣ Feedback Extraction Phase...")
            feedback_list = self.feedback_extractor.extract_feedback(test_case, failed_metrics)
            print(f"   📋 Extracted {len(feedback_list)} feedback items")
            
            # Phase 4: Prompt Enhancement
            print("4️⃣ Prompt Enhancement Phase...")
            enhanced_prompt = self.prompt_enhancer.enhance_prompt(current_prompt, feedback_list)
            
            # Phase 5: Re-execution
            print("5️⃣ Re-execution Phase...")
            enhanced_output = self._execute_prompt(enhanced_prompt, test_input)
            
            # Đánh giá output mới
            enhanced_test_case = LLMTestCase(
                input=test_input,
                actual_output=enhanced_output,
                context=[context] if context else None
            )
            
            enhanced_scores, enhanced_failed = self._evaluate_test_case(enhanced_test_case, metrics)
            
            # Tính improvement
            improvement = self._calculate_improvement(scores, enhanced_scores)
            
            # Lưu kết quả
            result = FeedbackResult(
                iteration=iteration,
                original_prompt=current_prompt if iteration == 1 else initial_prompt,
                enhanced_prompt=enhanced_prompt,
                original_output=output,
                enhanced_output=enhanced_output,
                original_scores=scores,
                enhanced_scores=enhanced_scores,
                improvement=improvement,
                feedback_used=feedback_list,
                timestamp=datetime.now().isoformat(),
                success=len(enhanced_failed) == 0
            )
            
            self.results.append(result)
            
            # In kết quả iteration
            self._print_iteration_summary(result)
            
            # Update prompt cho iteration tiếp theo
            current_prompt = enhanced_prompt
            
            if result.success:
                print("✅ Pipeline hoàn thành thành công!")
                return result
        
        print(f"⚠️ Đã đạt max iterations ({self.max_iterations})")
        return self.results[-1] if self.results else None
    
    def _execute_prompt(self, prompt: str, test_input: str) -> str:
        """Thực hiện prompt với input"""
        full_prompt = f"{prompt}\n\nInput: {test_input}\n\nOutput:"
        response = self.llm.invoke(full_prompt)
        return response.content.strip()
    
    def _evaluate_test_case(self, test_case: LLMTestCase, metrics: List[BaseMetric]) -> Tuple[Dict[str, float], List[BaseMetric]]:
        """Đánh giá test case với các metrics"""
        scores = {}
        failed_metrics = []
        
        for metric in metrics:
            try:
                score = metric.measure(test_case)
                scores[metric.__class__.__name__] = score
                
                if not metric.is_successful():
                    failed_metrics.append(metric)
            except Exception as e:
                logger.error(f"Error evaluating {metric.__class__.__name__}: {e}")
                scores[metric.__class__.__name__] = 0.0
                failed_metrics.append(metric)
        
        return scores, failed_metrics
    
    def _calculate_improvement(self, original_scores: Dict[str, float], enhanced_scores: Dict[str, float]) -> Dict[str, float]:
        """Tính toán mức cải thiện"""
        improvement = {}
        for metric_name in original_scores:
            if metric_name in enhanced_scores:
                improvement[metric_name] = enhanced_scores[metric_name] - original_scores[metric_name]
        return improvement
    
    def _print_iteration_summary(self, result: FeedbackResult):
        """In tóm tắt iteration"""
        print(f"\n📊 ITERATION {result.iteration} SUMMARY:")
        print(f"   ✅ Success: {result.success}")
        
        print("   📈 Improvements:")
        for metric, improvement in result.improvement.items():
            direction = "📈" if improvement > 0 else "📉" if improvement < 0 else "➡️"
            print(f"      {direction} {metric}: {improvement:+.3f}")
        
        print(f"   💬 Feedback items: {len(result.feedback_used)}")
        print("-" * 40)

print("✅ Automated Feedback Pipeline đã được tạo!")

## 🎯 Thực Hành: Complex Problem Solving với Feedback Loop

Bây giờ chúng ta sẽ test pipeline với một bài toán phức tạp.

In [ ]:
# Tạo test case phức tạp
complex_problem = """
Một công ty công nghệ có 200 nhân viên đang gặp vấn đề về hiệu suất làm việc từ xa. 
Khảo sát cho thấy: 60% nhân viên cảm thấy thiếu kết nối với đồng nghiệp, 
45% báo cáo khó khăn trong việc quản lý thời gian, và 30% nói rằng họ thiếu 
động lực. Chi phí vận hành văn phòng đã giảm 40% nhưng doanh thu giảm 15%.

Hãy phân tích vấn đề và đề xuất giải pháp toàn diện.
"""

# Prompt ban đầu (cố tình làm đơn giản để test feedback)
initial_prompt = """
Bạn là một chuyên gia tư vấn doanh nghiệp. Hãy phân tích vấn đề và đưa ra giải pháp.
Suy nghĩ từng bước một cách logic.
"""

# Tạo metrics để đánh giá
metrics = [
    LogicalFlowMetric(threshold=0.7),
    StepCompletenessMetric(threshold=0.8),
    AnswerRelevancyMetric(threshold=0.8)
]

print("🎯 Test case đã được tạo!")
print(f"Problem: {complex_problem[:100]}...")
print(f"Initial prompt: {initial_prompt[:100]}...")
print(f"Metrics: {[m.__class__.__name__ for m in metrics]}")

In [ ]:
# Chạy Feedback Pipeline
pipeline = AutomatedFeedbackPipeline(llm, max_iterations=3)

print("🚀 Bắt đầu chạy Automated Feedback Pipeline...\n")

final_result = pipeline.run_pipeline(
    initial_prompt=initial_prompt,
    test_input=complex_problem,
    metrics=metrics
)

print("\n" + "="*60)
print("🏁 PIPELINE HOÀN THÀNH!")
print("="*60)

## 📊 Phân Tích Kết Quả Pipeline

Hãy phân tích chi tiết kết quả của pipeline.

In [ ]:
# Phân tích kết quả chi tiết
def analyze_pipeline_results(pipeline: AutomatedFeedbackPipeline):
    """Phân tích chi tiết kết quả pipeline"""
    
    print("📊 PHÂN TÍCH KẾT QUẢ PIPELINE")
    print("="*50)
    
    if not pipeline.results:
        print("❌ Không có kết quả để phân tích")
        return
    
    # Tổng quan
    total_iterations = len(pipeline.results)
    final_success = pipeline.results[-1].success
    
    print(f"🔢 Tổng số iterations: {total_iterations}")
    print(f"✅ Kết quả cuối: {'Thành công' if final_success else 'Thất bại'}")
    print()
    
    # Phân tích từng iteration
    for i, result in enumerate(pipeline.results, 1):
        print(f"🔄 ITERATION {i}:")
        print(f"   📅 Timestamp: {result.timestamp}")
        print(f"   ✅ Success: {result.success}")
        
        # Scores
        print("   📊 Scores:")
        if result.original_scores:
            for metric, score in result.original_scores.items():
                enhanced_score = result.enhanced_scores.get(metric, score)
                improvement = result.improvement.get(metric, 0)
                print(f"      {metric}: {score:.3f} → {enhanced_score:.3f} ({improvement:+.3f})")
        else:
            for metric, score in result.enhanced_scores.items():
                print(f"      {metric}: {score:.3f}")
        
        # Feedback sử dụng
        print(f"   💬 Feedback items: {len(result.feedback_used)}")
        for j, feedback in enumerate(result.feedback_used[:2], 1):  # Chỉ hiển thị 2 feedback đầu
            print(f"      {j}. {feedback[:100]}...")
        
        print()
    
    # Phân tích improvement tổng thể
    if len(pipeline.results) > 1:
        print("📈 TỔNG KẾT IMPROVEMENT:")
        first_result = pipeline.results[0]
        last_result = pipeline.results[-1]
        
        for metric in first_result.enhanced_scores:
            if metric in last_result.enhanced_scores:
                initial_score = first_result.original_scores.get(metric, first_result.enhanced_scores[metric])
                final_score = last_result.enhanced_scores[metric]
                total_improvement = final_score - initial_score
                print(f"   {metric}: {initial_score:.3f} → {final_score:.3f} ({total_improvement:+.3f})")

# Chạy phân tích
analyze_pipeline_results(pipeline)

## 📋 So Sánh Prompt và Output

Hãy so sánh prompt và output ban đầu với kết quả cuối cùng.

In [ ]:
# So sánh prompt và output
def compare_prompts_and_outputs(pipeline: AutomatedFeedbackPipeline):
    """So sánh prompt và output qua các iterations"""
    
    if not pipeline.results:
        print("❌ Không có kết quả để so sánh")
        return
    
    print("🔍 SO SÁNH PROMPT VÀ OUTPUT")
    print("="*60)
    
    # Prompt evolution
    print("📝 EVOLUTION CỦA PROMPT:")
    print("-" * 30)
    
    for i, result in enumerate(pipeline.results, 1):
        print(f"\n🔄 ITERATION {i} PROMPT:")
        prompt_to_show = result.enhanced_prompt if i == 1 else result.enhanced_prompt
        print(f"   {prompt_to_show}")
        print()
    
    # Output comparison
    print("\n📄 SO SÁNH OUTPUT:")
    print("-" * 30)
    
    for i, result in enumerate(pipeline.results, 1):
        print(f"\n🔄 ITERATION {i}:")
        
        if result.original_output != "N/A":
            print(f"   📤 Original Output ({len(result.original_output)} chars):")
            print(f"   {result.original_output[:200]}...")
            print()
        
        print(f"   📥 Enhanced Output ({len(result.enhanced_output)} chars):")
        print(f"   {result.enhanced_output[:200]}...")
        print()
    
    # Final comparison
    if len(pipeline.results) > 1:
        first_result = pipeline.results[0]
        last_result = pipeline.results[-1]
        
        print("\n🏁 FINAL COMPARISON:")
        print("-" * 30)
        
        initial_output = first_result.original_output if first_result.original_output != "N/A" else first_result.enhanced_output
        final_output = last_result.enhanced_output
        
        print(f"📊 Length comparison: {len(initial_output)} → {len(final_output)} chars")
        print(f"📈 Length change: {len(final_output) - len(initial_output):+d} chars")

# Chạy so sánh
compare_prompts_and_outputs(pipeline)

## 🔍 Advanced Feedback Strategies

Bây giờ chúng ta sẽ triển khai các strategies nâng cao.

In [ ]:
class AdvancedFeedbackAnalyzer:
    """Phân tích feedback nâng cao với pattern recognition"""
    
    def __init__(self, llm):
        self.llm = llm
        self.failure_patterns = defaultdict(list)
        self.improvement_history = []
    
    def analyze_failure_patterns(self, pipeline_results: List[FeedbackResult]) -> List[FailurePattern]:
        """Phân tích patterns của failures"""
        
        print("🔍 ANALYZING FAILURE PATTERNS...")
        
        # Collect all feedback
        all_feedback = []
        for result in pipeline_results:
            all_feedback.extend(result.feedback_used)
        
        if not all_feedback:
            print("   ℹ️ Không có feedback để phân tích")
            return []
        
        # Categorize feedback
        categories = self._categorize_feedback(all_feedback)
        
        # Create failure patterns
        patterns = []
        for category, feedback_items in categories.items():
            if len(feedback_items) > 1:  # Pattern nếu xuất hiện > 1 lần
                pattern = FailurePattern(
                    pattern_type=category,
                    description=f"Recurring issue with {category.lower()}",
                    frequency=len(feedback_items),
                    suggested_fix=self._generate_pattern_fix(category, feedback_items),
                    examples=feedback_items[:3]  # Top 3 examples
                )
                patterns.append(pattern)
        
        # Print patterns
        self._print_failure_patterns(patterns)
        
        return patterns
    
    def _categorize_feedback(self, feedback_list: List[str]) -> Dict[str, List[str]]:
        """Phân loại feedback theo categories"""
        
        categories = defaultdict(list)
        
        # Define keywords for categories
        category_keywords = {
            "Logic Flow": ["logic", "flow", "sequence", "order", "reasoning", "step"],
            "Completeness": ["missing", "incomplete", "lack", "absent", "need more"],
            "Relevance": ["relevant", "related", "connection", "context", "topic"],
            "Structure": ["structure", "organization", "format", "layout", "arrange"],
            "Clarity": ["clear", "unclear", "confusing", "ambiguous", "vague"]
        }
        
        for feedback in feedback_list:
            feedback_lower = feedback.lower()
            categorized = False
            
            for category, keywords in category_keywords.items():
                if any(keyword in feedback_lower for keyword in keywords):
                    categories[category].append(feedback)
                    categorized = True
                    break
            
            if not categorized:
                categories["Other"].append(feedback)
        
        return dict(categories)
    
    def _generate_pattern_fix(self, category: str, feedback_items: List[str]) -> str:
        """Tạo suggested fix cho pattern"""
        
        fix_templates = {
            "Logic Flow": "Add explicit step-by-step reasoning with clear transitions",
            "Completeness": "Include comprehensive analysis covering all aspects",
            "Relevance": "Ensure all points directly address the main question",
            "Structure": "Use clear headings and organized sections",
            "Clarity": "Simplify language and provide clear explanations"
        }
        
        return fix_templates.get(category, "Review and improve based on feedback")
    
    def _print_failure_patterns(self, patterns: List[FailurePattern]):
        """In failure patterns"""
        
        if not patterns:
            print("   ✅ Không phát hiện failure patterns lặp lại")
            return
        
        print(f"   🔍 Phát hiện {len(patterns)} failure patterns:")
        print()
        
        for i, pattern in enumerate(patterns, 1):
            print(f"   {i}. 📊 {pattern.pattern_type}")
            print(f"      📈 Frequency: {pattern.frequency}")
            print(f"      📋 Description: {pattern.description}")
            print(f"      💡 Suggested fix: {pattern.suggested_fix}")
            print(f"      📝 Example: {pattern.examples[0][:100]}...")
            print()
    
    def generate_meta_feedback(self, pipeline_results: List[FeedbackResult]) -> str:
        """Tạo meta-feedback về toàn bộ pipeline"""
        
        if not pipeline_results:
            return "No results to analyze"
        
        meta_prompt = f"""
        Phân tích toàn bộ quá trình feedback loop với {len(pipeline_results)} iterations:
        
        RESULTS SUMMARY:
        {self._format_results_summary(pipeline_results)}
        
        Hãy đưa ra meta-feedback về:
        1. Hiệu quả của feedback loop
        2. Patterns trong quá trình cải thiện
        3. Điểm mạnh và điểm yếu
        4. Đề xuất để tối ưu pipeline
        
        Trả lời ngắn gọn, tập trung vào insights hữu ích.
        """
        
        response = self.llm.invoke(meta_prompt)
        return response.content.strip()
    
    def _format_results_summary(self, results: List[FeedbackResult]) -> str:
        """Format tóm tắt kết quả"""
        summary_lines = []
        
        for i, result in enumerate(results, 1):
            success_status = "✅" if result.success else "❌"
            avg_improvement = np.mean(list(result.improvement.values())) if result.improvement else 0
            summary_lines.append(
                f"Iteration {i}: {success_status} Success={result.success}, "
                f"Avg_Improvement={avg_improvement:.3f}, "
                f"Feedback_Count={len(result.feedback_used)}"
            )
        
        return "\n".join(summary_lines)

print("✅ Advanced Feedback Analyzer đã được tạo!")

In [ ]:
# Chạy advanced analysis
advanced_analyzer = AdvancedFeedbackAnalyzer(llm)

print("🔍 ADVANCED FEEDBACK ANALYSIS")
print("="*50)

# Phân tích failure patterns
failure_patterns = advanced_analyzer.analyze_failure_patterns(pipeline.results)

print("\n🧠 META-FEEDBACK:")
print("-" * 30)

# Tạo meta-feedback
meta_feedback = advanced_analyzer.generate_meta_feedback(pipeline.results)
print(meta_feedback)

## 📊 Performance Tracking và Visualization

Tạo dashboard để theo dõi performance qua các iterations.

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('default')

def create_performance_dashboard(pipeline_results: List[FeedbackResult]):
    """Tạo dashboard hiển thị performance"""
    
    if not pipeline_results:
        print("❌ Không có dữ liệu để visualize")
        return
    
    # Prepare data
    iterations = list(range(1, len(pipeline_results) + 1))
    
    # Collect metrics data
    metrics_data = defaultdict(list)
    success_data = []
    
    for result in pipeline_results:
        # Use enhanced scores (final scores for each iteration)
        for metric_name, score in result.enhanced_scores.items():
            metrics_data[metric_name].append(score)
        
        success_data.append(1 if result.success else 0)
    
    # Create subplots
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle('🚀 Feedback Pipeline Performance Dashboard', fontsize=16, fontweight='bold')
    
    # Plot 1: Metrics scores over iterations
    ax1 = axes[0, 0]
    for metric_name, scores in metrics_data.items():
        if len(scores) == len(iterations):
            ax1.plot(iterations, scores, marker='o', linewidth=2, label=metric_name)
    
    ax1.set_title('📈 Metrics Scores Over Iterations')
    ax1.set_xlabel('Iteration')
    ax1.set_ylabel('Score')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    ax1.set_ylim(0, 1)
    
    # Plot 2: Success rate
    ax2 = axes[0, 1]
    colors = ['red' if s == 0 else 'green' for s in success_data]
    ax2.bar(iterations, success_data, color=colors, alpha=0.7)
    ax2.set_title('✅ Success Rate by Iteration')
    ax2.set_xlabel('Iteration')
    ax2.set_ylabel('Success (1) / Failure (0)')
    ax2.set_ylim(0, 1.2)
    ax2.grid(True, alpha=0.3)
    
    # Plot 3: Improvement trends
    ax3 = axes[1, 0]
    
    improvement_data = defaultdict(list)
    for result in pipeline_results:
        for metric_name, improvement in result.improvement.items():
            improvement_data[metric_name].append(improvement)
    
    for metric_name, improvements in improvement_data.items():
        if len(improvements) == len(iterations):
            ax3.bar([i + 0.1 * list(improvement_data.keys()).index(metric_name) for i in iterations], 
                   improvements, width=0.15, label=metric_name, alpha=0.8)
    
    ax3.set_title('📊 Improvement per Iteration')
    ax3.set_xlabel('Iteration')
    ax3.set_ylabel('Improvement Score')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    ax3.axhline(y=0, color='black', linestyle='--', alpha=0.5)
    
    # Plot 4: Feedback count per iteration
    ax4 = axes[1, 1]
    feedback_counts = [len(result.feedback_used) for result in pipeline_results]
    ax4.bar(iterations, feedback_counts, color='purple', alpha=0.7)
    ax4.set_title('💬 Feedback Items per Iteration')
    ax4.set_xlabel('Iteration')
    ax4.set_ylabel('Number of Feedback Items')
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print summary statistics
    print("\n📊 PERFORMANCE SUMMARY:")
    print("-" * 40)
    
    final_success_rate = sum(success_data) / len(success_data)
    print(f"✅ Overall success rate: {final_success_rate:.1%}")
    
    if metrics_data:
        print("\n📈 Final Metrics Scores:")
        for metric_name, scores in metrics_data.items():
            if scores:
                initial_score = scores[0]
                final_score = scores[-1]
                improvement = final_score - initial_score
                print(f"   {metric_name}: {initial_score:.3f} → {final_score:.3f} ({improvement:+.3f})")
    
    total_feedback = sum(feedback_counts)
    avg_feedback = total_feedback / len(feedback_counts) if feedback_counts else 0
    print(f"\n💬 Total feedback items: {total_feedback}")
    print(f"💬 Average feedback per iteration: {avg_feedback:.1f}")

# Tạo dashboard
create_performance_dashboard(pipeline.results)

## 🎯 Real-World Case Study: Code Review Agent

Bây giờ chúng ta sẽ áp dụng feedback loop cho một use case thực tế - Code Review Agent.

In [ ]:
# Real-world case study: Code Review Agent
code_review_problem = """
Hãy review đoạn code Python sau và đưa ra feedback chi tiết:

```python
def process_user_data(data):
    users = []
    for item in data:
        if item['age'] > 18:
            user = {
                'name': item['name'],
                'email': item['email'],
                'age': item['age']
            }
            users.append(user)
    return users

def send_emails(users):
    import smtplib
    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.login('admin@company.com', 'password123')
    
    for user in users:
        message = f"Hello {user['name']}, welcome!"
        server.sendmail('admin@company.com', user['email'], message)
    
    server.quit()
```

Tập trung vào: security issues, error handling, code quality, performance.
"""

# Code review prompt (cố tình đơn giản)
code_review_initial_prompt = """
Bạn là một senior developer. Hãy review code và đưa ra feedback.
Hãy suy nghĩ từng bước một cách có hệ thống.
"""

# Custom metrics cho code review
class SecurityAnalysisMetric(BaseMetric):
    """Đánh giá khả năng phân tích security issues"""
    
    def __init__(self, threshold: float = 0.8):
        self.threshold = threshold
    
    def measure(self, test_case: LLMTestCase) -> float:
        security_keywords = [
            'password', 'hardcoded', 'credential', 'authentication',
            'security', 'vulnerability', 'sensitive', 'encryption',
            'sql injection', 'xss', 'csrf', 'validation'
        ]
        
        output_lower = test_case.actual_output.lower()
        security_mentions = sum(1 for keyword in security_keywords if keyword in output_lower)
        
        # Check if mentions hardcoded password specifically
        has_password_issue = 'password123' in test_case.actual_output or 'hardcoded' in output_lower
        
        # Score based on security awareness
        score = min(1.0, (security_mentions * 0.2) + (0.5 if has_password_issue else 0))
        
        self.score = score
        self.reason = f"Security analysis score: {score:.2f} (mentions: {security_mentions})"  
        self.success = score >= self.threshold
        return score
    
    def is_successful(self) -> bool:
        return self.success

class ErrorHandlingMetric(BaseMetric):
    """Đánh giá khả năng phát hiện thiếu error handling"""
    
    def __init__(self, threshold: float = 0.7):
        self.threshold = threshold
    
    def measure(self, test_case: LLMTestCase) -> float:
        error_keywords = [
            'try', 'except', 'error', 'exception', 'handle',
            'validation', 'check', 'null', 'none', 'empty'
        ]
        
        output_lower = test_case.actual_output.lower()
        error_mentions = sum(1 for keyword in error_keywords if keyword in output_lower)
        
        # Specific issues that should be caught
        catches_key_error = 'keyerror' in output_lower or 'key error' in output_lower
        mentions_validation = 'validation' in output_lower or 'validate' in output_lower
        
        score = min(1.0, (error_mentions * 0.15) + 
                   (0.3 if catches_key_error else 0) + 
                   (0.2 if mentions_validation else 0))
        
        self.score = score
        self.reason = f"Error handling analysis: {score:.2f} (mentions: {error_mentions})"
        self.success = score >= self.threshold
        return score
    
    def is_successful(self) -> bool:
        return self.success

# Code review metrics
code_review_metrics = [
    LogicalFlowMetric(threshold=0.7),
    StepCompletenessMetric(threshold=0.8),
    SecurityAnalysisMetric(threshold=0.8),
    ErrorHandlingMetric(threshold=0.7)
]

print("🎯 Code Review Case Study đã được setup!")
print(f"Problem: Code review for security and quality issues")
print(f"Metrics: {[m.__class__.__name__ for m in code_review_metrics]}")

In [ ]:
# Chạy Code Review Pipeline
code_review_pipeline = AutomatedFeedbackPipeline(llm, max_iterations=4)

print("🔍 CHẠY CODE REVIEW FEEDBACK PIPELINE")
print("="*60)

code_review_result = code_review_pipeline.run_pipeline(
    initial_prompt=code_review_initial_prompt,
    test_input=code_review_problem,
    metrics=code_review_metrics
)

print("\n" + "="*60)
print("🏁 CODE REVIEW PIPELINE HOÀN THÀNH!")
print("="*60)

In [ ]:
# Phân tích kết quả Code Review
print("📊 PHÂN TÍCH KẾT QUẢ CODE REVIEW")
print("="*50)

analyze_pipeline_results(code_review_pipeline)

print("\n📋 SO SÁNH CODE REVIEW OUTPUT:")
print("-" * 40)

if code_review_pipeline.results:
    final_result = code_review_pipeline.results[-1]
    
    print("🎯 FINAL CODE REVIEW OUTPUT:")
    print(final_result.enhanced_output)
    
    print("\n📈 FINAL SCORES:")
    for metric, score in final_result.enhanced_scores.items():
        status = "✅" if score >= 0.7 else "❌"
        print(f"   {status} {metric}: {score:.3f}")

In [ ]:
# Tạo dashboard cho Code Review Pipeline
print("📊 CODE REVIEW PERFORMANCE DASHBOARD")
print("="*50)

create_performance_dashboard(code_review_pipeline.results)

## 🔄 Adaptive Threshold Adjustment

Triển khai tính năng tự động điều chỉnh threshold dựa trên performance.

In [ ]:
class AdaptiveThresholdManager:
    """Quản lý adaptive threshold cho metrics"""
    
    def __init__(self, initial_thresholds: Dict[str, float]):
        self.initial_thresholds = initial_thresholds.copy()
        self.current_thresholds = initial_thresholds.copy()
        self.performance_history = defaultdict(list)
        self.adjustment_history = defaultdict(list)
    
    def update_thresholds(self, pipeline_results: List[FeedbackResult]) -> Dict[str, float]:
        """Cập nhật thresholds dựa trên performance history"""
        
        if not pipeline_results:
            return self.current_thresholds
        
        # Collect performance data
        for result in pipeline_results:
            for metric_name, score in result.enhanced_scores.items():
                self.performance_history[metric_name].append(score)
        
        # Adjust thresholds
        new_thresholds = {}
        
        for metric_name, scores in self.performance_history.items():
            if len(scores) < 2:
                new_thresholds[metric_name] = self.current_thresholds.get(metric_name, 0.7)
                continue
            
            # Calculate statistics
            avg_score = np.mean(scores)
            std_score = np.std(scores)
            recent_scores = scores[-3:]  # Last 3 scores
            recent_avg = np.mean(recent_scores)
            
            current_threshold = self.current_thresholds.get(metric_name, 0.7)
            
            # Adjustment logic
            if recent_avg > current_threshold + 0.1:  # Consistently performing well
                new_threshold = min(0.9, current_threshold + 0.05)  # Increase threshold
                adjustment_reason = "Performance consistently high"
            elif recent_avg < current_threshold - 0.1:  # Consistently underperforming
                new_threshold = max(0.3, current_threshold - 0.05)  # Decrease threshold
                adjustment_reason = "Performance consistently low"
            else:
                new_threshold = current_threshold  # Keep current
                adjustment_reason = "Performance stable"
            
            new_thresholds[metric_name] = new_threshold
            
            # Record adjustment
            if new_threshold != current_threshold:
                self.adjustment_history[metric_name].append({
                    'old_threshold': current_threshold,
                    'new_threshold': new_threshold,
                    'reason': adjustment_reason,
                    'avg_score': avg_score,
                    'recent_avg': recent_avg
                })
        
        self.current_thresholds = new_thresholds
        return new_thresholds
    
    def print_threshold_adjustments(self):
        """In lịch sử điều chỉnh threshold"""
        
        print("🎯 ADAPTIVE THRESHOLD ADJUSTMENTS")
        print("-" * 40)
        
        if not any(self.adjustment_history.values()):
            print("   ℹ️ Chưa có điều chỉnh threshold nào")
            return
        
        for metric_name, adjustments in self.adjustment_history.items():
            if adjustments:
                print(f"\n📊 {metric_name}:")
                for i, adj in enumerate(adjustments, 1):
                    direction = "📈" if adj['new_threshold'] > adj['old_threshold'] else "📉"
                    print(f"   {i}. {direction} {adj['old_threshold']:.3f} → {adj['new_threshold']:.3f}")
                    print(f"      Reason: {adj['reason']}")
                    print(f"      Recent avg: {adj['recent_avg']:.3f}")
        
        print("\n🎯 CURRENT THRESHOLDS:")
        for metric_name, threshold in self.current_thresholds.items():
            initial = self.initial_thresholds.get(metric_name, threshold)
            change = threshold - initial
            direction = "📈" if change > 0 else "📉" if change < 0 else "➡️"
            print(f"   {direction} {metric_name}: {initial:.3f} → {threshold:.3f} ({change:+.3f})")

# Test Adaptive Threshold Manager
initial_thresholds = {
    'LogicalFlowMetric': 0.7,
    'StepCompletenessMetric': 0.8,
    'SecurityAnalysisMetric': 0.8,
    'ErrorHandlingMetric': 0.7
}

threshold_manager = AdaptiveThresholdManager(initial_thresholds)

# Update với kết quả từ code review pipeline
updated_thresholds = threshold_manager.update_thresholds(code_review_pipeline.results)

# Print adjustments
threshold_manager.print_threshold_adjustments()

print("\n✅ Adaptive Threshold Manager hoàn thành!")

## 📚 Export và Lưu Trữ Kết Quả

Cuối cùng, chúng ta sẽ export kết quả để phân tích sau này.

In [ ]:
class FeedbackResultsExporter:
    """Export feedback results cho analysis và reporting"""
    
    def __init__(self):
        self.export_timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    def export_to_json(self, pipeline_results: List[FeedbackResult], filename: Optional[str] = None) -> str:
        """Export kết quả ra JSON file"""
        
        if not filename:
            filename = f"feedback_results_{self.export_timestamp}.json"
        
        # Convert to serializable format
        export_data = {
            'export_timestamp': self.export_timestamp,
            'total_iterations': len(pipeline_results),
            'overall_success': pipeline_results[-1].success if pipeline_results else False,
            'results': [asdict(result) for result in pipeline_results]
        }
        
        # Write to file
        filepath = f"data/{filename}"
        os.makedirs(os.path.dirname(filepath), exist_ok=True)
        
        with open(filepath, 'w', encoding='utf-8') as f:
            json.dump(export_data, f, ensure_ascii=False, indent=2)
        
        print(f"✅ Exported results to {filepath}")
        return filepath
    
    def export_to_csv(self, pipeline_results: List[FeedbackResult], filename: Optional[str] = None) -> str:
        """Export metrics data ra CSV"""
        
        if not filename:
            filename = f"feedback_metrics_{self.export_timestamp}.csv"
        
        # Prepare data for DataFrame
        rows = []
        
        for result in pipeline_results:
            base_row = {
                'iteration': result.iteration,
                'timestamp': result.timestamp,
                'success': result.success,
                'feedback_count': len(result.feedback_used)
            }
            
            # Add metric scores
            for metric_name, score in result.enhanced_scores.items():
                base_row[f'{metric_name}_score'] = score
            
            # Add improvements
            for metric_name, improvement in result.improvement.items():
                base_row[f'{metric_name}_improvement'] = improvement
            
            rows.append(base_row)
        
        # Create DataFrame and save
        df = pd.DataFrame(rows)
        filepath = f"data/{filename}"
        os.makedirs(os.path.dirname(filepath), exist_ok=True)
        
        df.to_csv(filepath, index=False)
        
        print(f"✅ Exported metrics to {filepath}")
        print(f"📊 Data shape: {df.shape}")
        
        return filepath
    
    def generate_summary_report(self, pipeline_results: List[FeedbackResult]) -> str:
        """Tạo summary report"""
        
        if not pipeline_results:
            return "No results to report"
        
        report_lines = [
            "🚀 FEEDBACK PIPELINE SUMMARY REPORT",
            "=" * 50,
            f"📅 Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}",
            f"🔄 Total Iterations: {len(pipeline_results)}",
            f"✅ Final Success: {pipeline_results[-1].success}",
            ""
        ]
        
        # Performance summary
        report_lines.extend([
            "📊 PERFORMANCE SUMMARY:",
            "-" * 30
        ])
        
        # Collect all metrics
        all_metrics = set()
        for result in pipeline_results:
            all_metrics.update(result.enhanced_scores.keys())
        
        for metric_name in sorted(all_metrics):
            scores = []
            for result in pipeline_results:
                if metric_name in result.enhanced_scores:
                    scores.append(result.enhanced_scores[metric_name])
            
            if scores:
                initial_score = scores[0]
                final_score = scores[-1]
                avg_score = np.mean(scores)
                improvement = final_score - initial_score
                
                direction = "📈" if improvement > 0 else "📉" if improvement < 0 else "➡️"
                report_lines.append(
                    f"   {direction} {metric_name}:"
                )
                report_lines.append(
                    f"      Initial: {initial_score:.3f} | Final: {final_score:.3f} | "
                    f"Avg: {avg_score:.3f} | Change: {improvement:+.3f}"
                )
        
        # Feedback analysis
        total_feedback = sum(len(result.feedback_used) for result in pipeline_results)
        avg_feedback = total_feedback / len(pipeline_results)
        
        report_lines.extend([
            "",
            "💬 FEEDBACK ANALYSIS:",
            "-" * 30,
            f"   Total feedback items: {total_feedback}",
            f"   Average per iteration: {avg_feedback:.1f}",
        ])
        
        # Success progression
        success_progression = [result.success for result in pipeline_results]
        success_rate = sum(success_progression) / len(success_progression)
        
        report_lines.extend([
            "",
            "✅ SUCCESS ANALYSIS:",
            "-" * 30,
            f"   Success rate: {success_rate:.1%}",
            f"   Success progression: {' → '.join('✅' if s else '❌' for s in success_progression)}"
        ])
        
        return "\n".join(report_lines)

# Export results
exporter = FeedbackResultsExporter()

print("📤 EXPORTING FEEDBACK RESULTS")
print("=" * 40)

# Export JSON
json_file = exporter.export_to_json(code_review_pipeline.results)

# Export CSV
csv_file = exporter.export_to_csv(code_review_pipeline.results)

# Generate and print report
print("\n📋 SUMMARY REPORT:")
print("-" * 40)
summary_report = exporter.generate_summary_report(code_review_pipeline.results)
print(summary_report)

# Save report to file
report_filename = f"data/feedback_report_{exporter.export_timestamp}.txt"
with open(report_filename, 'w', encoding='utf-8') as f:
    f.write(summary_report)

print(f"\n✅ Report saved to {report_filename}")

## 🎓 Tổng Kết và Bài Tập

### 📚 Kiến Thức Đã Học

Trong notebook này, chúng ta đã học:

1. **Automated Feedback Pipeline**: 5-phase execution cycle
2. **Advanced Feedback Strategies**: Pattern recognition, meta-feedback
3. **Adaptive Thresholds**: Tự động điều chỉnh dựa trên performance
4. **Real-world Applications**: Code Review Agent use case
5. **Performance Tracking**: Visualization và analysis
6. **Export và Reporting**: Lưu trữ kết quả cho phân tích sau

### 🎯 Bài Tập Thực Hành

1. **Tùy chỉnh Metrics**: Tạo custom metrics cho domain cụ thể
2. **Multi-agent Evaluation**: Áp dụng cho complex agent workflows
3. **Production Integration**: Tích hợp vào CI/CD pipeline
4. **A/B Testing**: So sánh different feedback strategies

### 🚀 Ứng Dụng Thực Tiễn

Framework này có thể áp dụng cho:
- **Code Review Automation**
- **Content Generation Quality Control**
- **Customer Support Bot Improvement**
- **Educational Content Assessment**
- **Legal Document Analysis**

---

**🎉 Chúc mừng!** Bạn đã hoàn thành series DeepEval comprehensive framework!

Tiếp theo, hãy áp dụng những kiến thức này vào dự án thực tế của bạn.

In [ ]:
# Final demonstration - Mini exercise
print("🎯 MINI EXERCISE: Tạo Feedback Loop cho Domain của Bạn")
print("=" * 60)
print("""
📝 CHALLENGE:
Hãy tạo một feedback loop cho một use case cụ thể trong domain của bạn.

STEPS:
1. Xác định problem domain (VD: Email writing, SQL query generation, etc.)
2. Tạo 2-3 custom metrics phù hợp
3. Thiết kế initial prompt (cố tình đơn giản)
4. Chạy feedback pipeline
5. Phân tích kết quả

TỪ KHÓA QUAN TRỌNG:
- Domain-specific metrics
- Iterative improvement
- Performance tracking
- Real-world applicability

💡 TIP: Bắt đầu với use case đơn giản, sau đó mở rộng complexity!
""")

print("\n🚀 Bạn đã sẵn sàng áp dụng DeepEval vào dự án thực tế!")
print("✨ Good luck và happy coding! ✨")